In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers

In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/train_qa.csv'
df = pd.read_csv(path)

df['input_text'] = df['story'] + " [SEP] " + df['question']


df['label'] = df['answer'].map({'yes': 1, 'no': 0})

print(f"Data Loaded: {len(df)} rows")
df.head()

Data Loaded: 10000 rows


,story,question,answer,input_text,label
0,Mary moved to the bathroom . Sandra journeyed ...,Is Sandra in the hallway ?,no,Mary moved to the bathroom . Sandra journeyed ...,0
1,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bathroom ?,no,Mary moved to the bathroom . Sandra journeyed ...,0
2,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the office ?,no,Mary moved to the bathroom . Sandra journeyed ...,0
3,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes,Mary moved to the bathroom . Sandra journeyed ...,1
4,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes,Mary moved to the bathroom . Sandra journeyed ...,1


In [ ]:
from transformers import DistilBertTokenizer
import tensorflow as tf

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


def encode_data(texts):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="tf"
    )

encoded_inputs = encode_data(df['input_text'])
labels = tf.convert_to_tensor(df['label'].values)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf

model = TFDistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    use_safetensors=False
)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
train_features = {
    'input_ids': encoded_inputs['input_ids'],
    'attention_mask': encoded_inputs['attention_mask']
}

print("Starting training...")
model.fit(
    train_features,
    labels,
    epochs=3,
    batch_size=8
)

tf_model.h5:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_39', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Starting training...
Epoch 1/3
1250/1250 [==============================] - 199s 137ms/step - loss: 0.5038 - accuracy: 0.7401
Epoch 2/3
1250/1250 [==============================] - 172s 138ms/step - loss: 0.3546 - accuracy: 0.8479
Epoch 3/3
1250/1250 [==============================] - 172s 138ms/step - loss: 0.3091 - accuracy: 0.8707


In [ ]:
import numpy as np

def predict_answer(story, question):
    
    text = story + " [SEP] " + question
    inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True, max_length=256)

   
    outputs = model(inputs)

    
    prediction = tf.argmax(outputs.logits, axis=-1).numpy()[0]

    return "Yes" if prediction == 1 else "No"


test_story = "John moved to the garden. Sandra went to the office. John journeyed to the kitchen."
test_question = "Is John in the kitchen?"

print(f"Question: {test_question}")
print(f"Model Prediction: {predict_answer(test_story, test_question)}")

Question: Is John in the kitchen?
Model Prediction: Yes


In [8]:
test_cases = [
    ("Daniel went to the office. Mary moved to the garden.", "Is Daniel in the office?"),
    ("Sandra went to the bathroom. Sandra moved to the kitchen.", "Is Sandra in the bathroom?"),
    ("John moved to the hallway. Mary moved to the office. Daniel is in the kitchen.", "Is Mary in the kitchen?"),
    ("Mary moved to the bedroom. John went to the garden.", "Is John in the garden?")
]

for story, question in test_cases:
    result = predict_answer(story, question)
    print(f"Story: {story}")
    print(f"Q: {question} -> AI Answer: {result}\n")

Story: Daniel went to the office. Mary moved to the garden.
Q: Is Daniel in the office? -> AI Answer: Yes

Story: Sandra went to the bathroom. Sandra moved to the kitchen.
Q: Is Sandra in the bathroom? -> AI Answer: No

Story: John moved to the hallway. Mary moved to the office. Daniel is in the kitchen.
Q: Is Mary in the kitchen? -> AI Answer: No

Story: Mary moved to the bedroom. John went to the garden.
Q: Is John in the garden? -> AI Answer: Yes



In [9]:
# Create a folder for the model
import os
save_directory = "./neuroqa_transformer_v2"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Save everything
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and Tokenizer saved to {save_directory}")

Model and Tokenizer saved to ./neuroqa_transformer_v2


In [10]:
from google.colab import files
import shutil

# Zip the folder first
shutil.make_archive("neuroqa_model", 'zip', save_directory)

# Download the zip file
files.download("neuroqa_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>